In [1]:
import pathlib
import sys
import warnings

import autograd
import autograd.numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import autocrit

In [2]:
sys.path.append("..")

import shared.format
import shared.tools

In [3]:
plt.rcParams["font.size"] = shared.format.FONTSIZE

In [4]:
pd.options.mode.chained_assignment = None

warnings.filterwarnings(
    "ignore", category=FutureWarning,
    message="Using a non-tuple sequence for multidimensional indexing is deprecated;",
    module="autograd")

In [5]:
import autocrit

In [6]:
import autocrit_tools
import autocrit_tools.linearpaper.utils as utils

In [7]:
pickle_dir = pathlib.Path("data") / "linear-results"

In [8]:
def compute_index(spectrum, eigval_cutoff=1e-5):
    return np.mean(spectrum < -eigval_cutoff)

In [9]:
cutoffs = [1e-10, 1e-6, 1e-4, np.inf]
cutoff_strings = [r"$\varepsilon=$1e-10", r"$\varepsilon=$1e-6", r"$\varepsilon=$1e-4", "no cutoff"]

In [10]:
cutoff_fnames = ["1e-10", "1e-6", "1e-4", "none"]

cutoff_cp_df_names= [pickle_dir / ("cutoff_" + cutoff_fname + ".pkl")
                     for cutoff_fname in cutoff_fnames]

In [11]:
import autocrit_tools.path as path
import autocrit_tools.dataframes as dataframes

In [12]:
def clean_path_columns(df, path_columns, split_str, new_prefix):
    for path_column in path_columns:
        df[path_column] = df[path_column].apply(clean_path, args=(split_str, new_prefix))
        
def clean_path(path, split_str, new_prefix):
    path_end = path.split(split_str)[-1]
    return os.path.join(new_prefix, path_end)

def add_index_sets(cp_df):
    maps = cp_df.final_theta.apply(
        lambda theta: utils.theta_to_map(theta, NETWORK))
    index_sets = [utils.map_to_index_set(_map) for _map in maps]

    cp_df["index_set"] = index_sets

In [13]:
root_dir = "data/"

In [60]:
data_ID = "linear-results"
network_ID = "dlae"

network_paths = path.ExperimentPaths(data_ID=data_ID, network_ID=network_ID, root=root_dir)

target_optimization_ID = "gd_optimizer_fimeXE"

In [61]:
optimization_df = dataframes.construct_experiments_df(
    network_paths.network_dir)

target_optimization_row = optimization_df.loc[target_optimization_ID]

data, NETWORK, _ = dataframes.reconstruct_from_row(
    target_optimization_row)

In [62]:
optimizer_paths = path.ExperimentPaths.from_optimizer_dir(target_optimization_row.optimizer_dir)

In [63]:
critfinders_dir = optimizer_paths.optimizer_dir

cf_df = dataframes.construct_experiments_df(
    critfinders_dir, experiment_type="critfinder")

In [64]:
expanded_cf_df = pd.DataFrame.from_dict(
    dict(cf_df.finder_kwargs_finder)
    ).transpose().join(cf_df)

In [66]:
def select_critfinders(cf_df, method_str, theta_perturbs=None, init_theta="uniform_f", 
                       min_runs=0, max_runs=1000):
    
    selector = cf_df.index.str.startswith(method_str) &\
               cf_df.init_theta_experiment.str.endswith(init_theta)
    
    if theta_perturbs is None:
        selector = selector & cf_df.theta_perturb_experiment.isnull()
    else:
        selector = selector & cf_df.theta_perturb_experiment.isin(theta_perturbs)
    
    cf_ids = expanded_cf_df.index[selector]
    
    cp_dfs =  utils.make_cp_dfs(cf_ids, cf_df)
    
    try:
        cp_dfs, cf_ids = zip(*[(cp_df, cf_id) for cp_df, cf_id in zip(cp_dfs, cf_ids)
                               if (len(cp_df) >= min_runs) & (len(cp_df) <= max_runs)])
    except ValueError:
        cp_dfs = cf_ids = []
    
    
    return cp_dfs, list(cf_ids)

In [67]:
def to_candidate_cp(cp_df, network, return_failures=False, cp_cutoff=1e-10):
    
    if return_failures:
        candidate_cp_df, failures_df = utils.filter_to_candidate_cps(
            cp_df, network, cp_cutoff=cp_cutoff, return_failures=True)
        candidate_cp_df["morse_index"] = utils.get_hessian_info(
            candidate_cp_df.candidate_theta, network)[-1]
        return candidate_cp_df, failures_df
    else:
        candidate_cp_df = utils.filter_to_candidate_cps(
            cp_df, network, cp_cutoff=cp_cutoff, return_failures=False)
        candidate_cp_df["morse_index"] = utils.get_hessian_info(
            candidate_cp_df.candidate_theta, network)[-1]
        
        return candidate_cp_df

# Performance Comparison Figure

In [44]:
MR_cp_dfs, MR_cf_ids = select_critfinders(expanded_cf_df,
                                          method_str="newtonMR",
                                         theta_perturbs=[None, -2])

In [22]:
TR_cp_dfs, TR_cf_ids = select_critfinders(expanded_cf_df,
                                          method_str="newtonTR",
                                         theta_perturbs=[None, -2])

In [68]:
gnm_cp_dfs, gnm_cf_ids = select_critfinders(expanded_cf_df,
                                          method_str="gnm",
                                         theta_perturbs=[None, -2])

In [45]:
merged_MR_cp_df = pd.concat(MR_cp_dfs)

In [23]:
merged_TR_cp_df = pd.concat(TR_cp_dfs)

In [70]:
merged_gnm_cp_df = pd.concat(gnm_cp_dfs)

In [ ]:
merged_MR_candidate_cp_df, merged_MR_failures_df = to_candidate_cp(
    merged_MR_cp_df, NETWORK,
    return_failures=True, cp_cutoff=1e-10)

In [40]:
merged_TR_candidate_cp_df, merged_TR_failures_df = to_candidate_cp(
    merged_TR_cp_df, NETWORK,
    return_failures=True, cp_cutoff=1e-10)

In [72]:
merged_gnm_candidate_cp_df, merged_gnm_failures_df = to_candidate_cp(
    merged_gnm_cp_df, NETWORK,
    return_failures=True, cp_cutoff=1e-10)

In [49]:
def make_slim(cp_df):
    slim_df = cp_df.copy()
    columns_to_drop = [
        "log_kwargs", "alpha", "beta", "rho", "check_pure", "rho_pure",
        "log_mrqlp", "rtol", "maxit", "thetas", "run_length", "alphas",
        "step_size", "gammas", "pure_accepted"]
    for column in columns_to_drop:
        try:
            slim_df = slim_df.drop(axis="columns", labels=[column])
        except KeyError:
            continue
    return slim_df

In [50]:
slim_TR_candidate_df = make_slim(merged_TR_candidate_cp_df)
slim_TR_failures_df = make_slim(merged_TR_failures_df)

slim_TR_candidate_df.to_pickle("data/linear-results/newtonTR-success.pkl")
slim_TR_failures_df.to_pickle("data/linear-results/newtonTR-failure.pkl")

In [51]:
slim_MR_candidate_df = make_slim(merged_MR_candidate_cp_df)
slim_MR_failures_df = make_slim(merged_MR_failures_df)

slim_MR_candidate_df.to_pickle("data/linear-results/newtonMR-success.pkl")
slim_MR_failures_df.to_pickle("data/linear-results/newtonMR-failure.pkl")

In [73]:
slim_gnm_candidate_df = make_slim(merged_gnm_candidate_cp_df)
slim_gnm_failures_df = make_slim(merged_gnm_failures_df)

slim_gnm_candidate_df.to_pickle("data/linear-results/gnm-success.pkl")
slim_gnm_failures_df.to_pickle("data/linear-results/gnm-failure.pkl")

# Cutoffs Figure

In [ ]:
def make_cutoff_cp_df(cp_df, network, cp_cutoff):
    cutoff_cp_df, failures_df = utils.filter_to_candidate_cps(
        cp_df, network, cp_cutoff=cp_cutoff, return_failures=True, cut_early=True)
    cutoff_cp_df["morse_index"] = utils.get_hessian_info(
        cutoff_cp_df.candidate_theta, network)[-1]
    return cutoff_cp_df

In [ ]:
cutoff_cp_dfs = [make_cutoff_cp_df(merged_MR_cp_df, NETWORK, cutoff) for cutoff in cutoffs]

In [ ]:
slim_cutoff_cp_dfs = [make_slim(cutoff_cp_df) for cutoff_cp_df in cutoff_cp_dfs]

In [ ]:
[cutoff_cp_df.to_pickle(cutoff_cp_df_name)
 for cutoff_cp_df, cutoff_cp_df_name in zip(cutoff_cp_dfs,cutoff_cp_df_names)];

In [ ]:
import autocrit_tools.linearpaper.panels as panels

In [ ]:
panels.make_trajectory_panel(merged_MR_candidate_cp_df,
                             merged_MR_failures_df);
plt.xscale("linear"); plt.xlim(0, 500);

In [ ]:
panels.make_loss_index_panel(merged_MR_candidate_cp_df,
                             analytical_cp_df);

# Nonlinear-Failure Figure

In [57]:
nonlinear_failure_candidate_cp_df = pd.read_pickle(
    "data/nonlinear-results/dnn-failure-full-success.pkl")
nonlinear_failure_failures_df = pd.read_pickle(
    "data/nonlinear-results/dnn-failure-full-failure.pkl")

In [59]:
slim_nonlinear_failure_candidate_df = make_slim(nonlinear_failure_candidate_cp_df)
slim_nonlinear_failure_failures_df = make_slim(nonlinear_failure_failures_df)

slim_nonlinear_failure_candidate_df.to_pickle("data/nonlinear-results/dnn-failure-success.pkl")
slim_nonlinear_failure_failures_df.to_pickle("data/nonlinear-results/dnn-failure-failure.pkl")